In [29]:
import json
import pandas as pd
import numpy as np
import ast
import math 

from nltk.parse.generate import generate, demo_grammar
from nltk import CFG

def remove_second_pair(list_orders):
    removed_second_pairs=[]
    for item in list_orders:
        removed_second_pairs.append(item[0])
    return(removed_second_pairs)


def add_words_lists(words,list_orders):
    table = pd.concat([words,list_orders],axis=1)
    new_list_orders=[]
        
    for row in range(0,len(table)):
        if (table.iloc[row,0]!=np.NaN) and (table.iloc[row,1]!=[]):            
            if table.iloc[row,0] in table.iloc[row,1]:
                table.iloc[row,1].remove(table.iloc[row,0])
            table.iloc[row,1].insert(0,table.iloc[row,0])

    return (table.iloc[:,1])
        
def convert_list_2_cfg(ordinary_list):
    acc_str=""
    for w in ordinary_list:
        acc_str+=" | "+ "\""+w+"\""
    return(acc_str)    
    

def generate_cfg(wpos,w1,w2,w3,s1,s2,s3,trigger_rule,group):
    cfg_rules=[]

    for idx in range(0,len(wpos)):
        wpos[idx][1]=wpos[idx][1].replace(" ", "_")
        wpos[idx][1]=wpos[idx][1].capitalize()+str(idx+1)


    str_start="S -> "
    for idx in range(0,len(wpos)):
        str_start +=" "+wpos[idx][1]
    str_start +=" Trigger_Rule"

    cfg_rules.append(str_start)  
    


    for idx in range(0,len(wpos)):
        str_rule=""
        str_rule +=wpos[idx][1]+" -> "+"\""+wpos[idx][0]+"\""
        cfg_rules.append(str_rule)  


    if not isinstance(w1, str):
        if np.isnan(w1):
            w1=""
            
    if not isinstance(w2, str):
        if np.isnan(w2):
            w2=""
            
    if not isinstance(w3, str):
        if np.isnan(w3):
            w3=""            

    
    cfg_rules_augmented=[]    
    for rule in cfg_rules:
        if rule.find(w1)>=0:
            cfg_rules_augmented.append(rule+convert_list_2_cfg(s1[1:]))
        elif rule.find(w2)>=0:
            cfg_rules_augmented.append(rule+convert_list_2_cfg(s2[1:]))
        elif rule.find(w3)>=0:
            cfg_rules_augmented.append(rule+convert_list_2_cfg(s3[1:]))
        else:
            cfg_rules_augmented.append(rule)


    cfg_grammar=""
    for rule in cfg_rules_augmented:
        cfg_grammar+=rule+"\n"
    
    cfg_grammar+="Trigger_Rule -> "
    for tr in trigger_rule:
        cfg_grammar+="\"|"+tr+"|Group"+str(group)+"\" |"
    cfg_grammar=cfg_grammar[0:-1]
    cfg_grammar+="\n"
    
    return(cfg_grammar)      
   

In [31]:
words_table=pd.read_excel("./excels/4_Phase3_TranslationTable.xlsx")  
words_table=words_table.reset_index()
words_table=words_table.fillna("")

exp_fr=words_table["French Expression"]
exp_en=words_table["English Expression(s)"].apply(lambda s: ast.literal_eval(s))

indexes=words_table["Occured Index(es)"].apply(lambda s: ast.literal_eval(s))
groups=words_table["Occured Groups"].apply(lambda s: ast.literal_eval(s))
rules=words_table["Rule"].apply(lambda s: ast.literal_eval(s))

word_pos=words_table["Tokenization with POS"].apply(lambda s: ast.literal_eval(s))


word1=words_table["word1"]
syn1_list=words_table["syn_1"].apply(lambda s: ast.literal_eval(s))


word2=words_table["word2"]
syn2_list=words_table["syn_2"].apply(lambda s: ast.literal_eval(s))


word3=words_table["word3"]
syn3_list=words_table["syn_3"].apply(lambda s: ast.literal_eval(s))





json_notebook = {}  

json_notebook["metadata"] = {}
json_notebook["metadata"]["kernelspec"]={}
json_notebook["metadata"]["kernelspec"]["display_name"]= "Python 3"
json_notebook["metadata"]["kernelspec"]["language"]= "python"
json_notebook["metadata"]["kernelspec"]["name"]= "python3"

json_notebook["metadata"]["language_info"]={}
json_notebook["metadata"]["language_info"]["codemirror_mode"]={}
json_notebook["metadata"]["language_info"]["codemirror_mode"]["name"]= "ipython"
json_notebook["metadata"]["language_info"]["codemirror_mode"]["version"]= 3

json_notebook["metadata"]["file_extension"]= ".py"
json_notebook["metadata"]["mimetype"]= "text/x-python"
json_notebook["metadata"]["name"]= "python"
json_notebook["metadata"]["nbconvert_exporter"]= "python"
json_notebook["metadata"]["pygments_lexer"]= "ipython3"
json_notebook["metadata"]["version"]= "3.7.2"


json_notebook["nbformat"]= 4
json_notebook["nbformat_minor"]=2



seperation_margin=[[0,100],[101,200],[201,300],[301,400],[401,500],[501,600],[601,737]]


for sm in seperation_margin:
    
    json_notebook["cells"]=[]
    
    index_start=sm[0]
    index_end=sm[1]
    
    strlist="_"+str(index_start)+"_"+str(index_end)
    
    threshold=15
    
    for i in range(index_start,index_end+1): 
        print(i)
        if (i!=661 and i!=245):
            expfr=exp_fr[i]
            expen=exp_en[i]
            wpos=word_pos[i]
            w1=word1[i]
            w2=word2[i]
            w3=word3[i]
            s1=syn1_list[i][0:threshold]
            s2=syn2_list[i][0:threshold]
            s3=syn3_list[i][0:threshold]
            index_r=indexes[i]
            group=groups[i]
            rule=rules[i]

            cfg_grammar=generate_cfg(wpos,w1,w2,w3,s1,s2,s3,rule,group)
            print(cfg_grammar)


            json_cell={}
            json_cell["cell_type"] = "code"
            json_cell["metadata"] = {}
            json_cell["outputs"] = []
            json_cell["execution_count"] = "null"
            json_source1=["from nltk.parse.generate import generate, demo_grammar\n",
                "from nltk import CFG\n",
                "\n",
                "#  Item Number     : "+str(i)+"\n",
                "#  French Term     : "+str(expfr)+"\n",
                "#  English Term(s) : "+str(expen)+"\n",
                "#  Index(es)       : "+str(index_r)+"\n",
                "#  Grouping(s)     : "+str(group)+"\n\n",
                "\n",
                "cfg_grammar= \"\"\"\n"]
            json_source2=[cfg_grammar]
            json_source3=["\"\"\"\n",
                "\n",
                "for sentence in generate(CFG.fromstring(cfg_grammar), n=1000):\n",
                "    print(' '.join(sentence))"]
            json_cell["source"]=json_source1+json_source2+json_source3

            json_notebook["cells"].append(json_cell)



    with open("./notebooks/Validation_Notebook"+strlist+".ipynb", 'w') as outfile:  
        json.dump(json_notebook, outfile)
print("Finished...")   


0
S ->  Adjective1 Noun2 Numeral3 Noun4 Adposition5 Trigger_Rule
Adjective1 -> "\"
Noun2 -> ">"
Numeral3 -> "0"
Noun4 -> "année"
Adposition5 -> "de"
Trigger_Rule -> "|forward|trigger|historical|30|Group[2]" 

1
S ->  Adjective1 Noun2 Numeral3 Adposition4 Trigger_Rule
Adjective1 -> "\"
Noun2 -> ">"
Numeral3 -> "0-mois"
Adposition4 -> "de"
Trigger_Rule -> "|forward|trigger|historical|30|Group[2]" 

2
S ->  Adjective1 Noun2 Numeral3 Punctuation4 Noun5 Adposition6 Trigger_Rule
Adjective1 -> "\"
Noun2 -> ">"
Numeral3 -> "0"
Punctuation4 -> "-"
Noun5 -> "années"
Adposition6 -> "de"
Trigger_Rule -> "|forward|trigger|historical|30|Group[2]" 

3
S ->  Adjective1 Noun2 Numeral3 Punctuation4 Noun5 Adposition6 Trigger_Rule
Adjective1 -> "\"
Noun2 -> ">"
Numeral3 -> "0"
Punctuation4 -> "-"
Noun5 -> "année"
Adposition6 -> "de"
Trigger_Rule -> "|forward|trigger|historical|30|Group[2]" 

4
S ->  Adjective1 Noun2 Numeral3 Punctuation4 Noun5 Adposition6 Trigger_Rule
Adjective1 -> "\"
Noun2 -> ">"
Numera


386
S ->  Adverb1 Trigger_Rule
Adverb1 -> "doux"
Trigger_Rule -> "|forward|termination|negated|10|Group[666]" 

387
S ->  Noun1 Auxiliary2 Auxiliary3 Trigger_Rule
Noun1 -> "maman" | "mamma" | "mamelle" | "accouchée" | "mama" | "chrysanthemum morifolium" | "procréatrice" | "génitrice" | "Mama" | "maman"
Auxiliary2 -> "a"
Auxiliary3 -> "appelé"
Trigger_Rule -> "|both|termination|historical|30|Group[668, 669]" |"|both|termination|nonpatient|30|Group[668, 669]" 

388
S ->  Noun1 Trigger_Rule
Noun1 -> "maman" | "mamma" | "mamelle" | "accouchée" | "mama" | "chrysanthemum morifolium" | "procréatrice" | "génitrice" | "Mama" | "maman"
Trigger_Rule -> "|forward|trigger|nonpatient|30|Group[670]" 

389
S ->  Determiner1 Noun2 Trigger_Rule
Determiner1 -> "la"
Noun2 -> "maman" | "mamma" | "mamelle" | "accouchée" | "mama" | "chrysanthemum morifolium" | "procréatrice" | "génitrice" | "Mama" | "maman"
Trigger_Rule -> "|forward|trigger|nonpatient|30|Group[670]" 

390
S ->  Verb1 Determiner2 Adjective3 

601
S ->  Noun1 Adposition2 Noun3 Trigger_Rule
Noun1 -> "rapports" | "liaison" | "commerce" | "connexion" | "contact" | "analogie" | "affinité" | "similitude" | "union" | "correspondance"
Adposition2 -> "de"
Noun3 -> "soeur" | "novice" | "nonne" | "mère" | "frangine" | "béguine" | "moniale" | "fille" | "religieuse" | "congréganiste"
Trigger_Rule -> "|both|termination|historical|30|Group[1034, 1035]" |"|both|termination|nonpatient|30|Group[1034, 1035]" 

602
S ->  Noun1 Auxiliary2 Verb3 Trigger_Rule
Noun1 -> "soeur" | "novice" | "nonne" | "mère" | "frangine" | "béguine" | "moniale" | "fille" | "religieuse" | "congréganiste"
Auxiliary2 -> "a"
Verb3 -> "rapporté" | "rendre" | "dénoncer" | "porter" | "remettre" | "produire" | "ramener" | "citer" | "dire" | "apporter"
Trigger_Rule -> "|both|termination|historical|30|Group[1034, 1035]" |"|both|termination|nonpatient|30|Group[1034, 1035]" 

603
S ->  Noun1 Auxiliary2 Auxiliary3 Trigger_Rule
Noun1 -> "soeur" | "novice" | "nonne" | "mère" | "fr


725
S ->  Adjective1 Adposition2 Trigger_Rule
Adjective1 -> "inquiet"
Adposition2 -> "pour"
Trigger_Rule -> "|forward|trigger|uncertain|30|Group[1285]" 

726
S ->  Verb1 Adposition2 Trigger_Rule
Verb1 -> "inquiétant" | "alarmant" | "angoissant" | "troublant" | "sinistre" | "redoutable" | "louche" | "menaçant" | "sombre" | "terrifiant"
Adposition2 -> "pour"
Trigger_Rule -> "|forward|trigger|uncertain|30|Group[1285]" 

727
S ->  Verb1 Trigger_Rule
Verb1 -> "donné" | "déterminé" | "fixé" | "offert" | "foutre" | "fixer" | "porter" | "certain" | "attribué" | "dire"
Trigger_Rule -> "|forward|termination|uncertain|4|Group[1288, 1289]" |"|forward|termination|conditional|4|Group[1288, 1289]" 

728
S ->  Adposition1 Noun2 Adposition3 Trigger_Rule
Adposition1 -> "avec"
Noun2 -> "résolution" | "fermeté" | "énergie" | "volonté" | "annulation" | "détermination" | "intention" | "dessein" | "courage" | "fin"
Adposition3 -> "de"
Trigger_Rule -> "|forward|trigger|negated|5|Group[1290]" 

729
S ->  Noun